# Wrangling DataFrames with Pandas    
## Author: Erika Duan    

![](../02_figures/04_manipulating-pandas-header.jpg)

# Dataset background  

We will be using open government data to familiarise ourselves with Pandas DataFrame manipulations. The dataset we will use describes the [index of disadvantage by schools in South Australia](https://data.gov.au/dataset/ds-sa-4ab8a539-eab5-48e2-8d88-a599d6114126/details?q=school), with a focus on [2020 data](https://data.gov.au/dataset/ds-sa-4ab8a539-eab5-48e2-8d88-a599d6114126/distribution/dist-sa-86d35483-feff-42b5-ac05-ad3186ac39de/details?q=school).     

+ The index is a socio-economic index, used by the South Australian Department for Education to allocate resources to schools to address educational disadvantage related to socio-economic status.  
+ The most disadvantaged schools have an index of 1.    
+ The least disadvantaged have an index of 7.   
+ Data is currently available for 2017, 2018 and 2020.  

# Loading data     

We can load data in two different ways:  

1. Locally using the `pathlib` library and the Pandas `read_csv` function.  
2. Directly through the data.gov.au [CKAN data API](https://docs.ckan.org/en/latest/maintaining/datastore.html).    

**Note:** A guide to querying open data via CKAN can be found [here](https://alan-y.netlify.app/post/ckan-api/#querying-ckan).  

In [67]:
#-----method 1: load csv file locally-----  
import pathlib
import pandas as pd     

working_dir = pathlib.Path.cwd() # get current working directory  
data_path = working_dir.parent / '01_raw_data' / '04_index-of-disadvantage-by-school-2020.csv'

raw_data = pd.read_csv(data_path)  
raw_data.head(3)

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [68]:
#-----method 2: access data as JSON file through CKAN data API-----  
import pathlib
import numpy as np
import pandas as pd
import requests
import json

url = "https://data.sa.gov.au/data/api/3/action/datastore_search?resource_id=86d35483-feff-42b5-ac05-ad3186ac39de" 
json_query = requests.get(url).json() # returns a dictionary object 

json_content = json.dumps(json_query, indent = 1, sort_keys = False) 
json_df = pd.read_json(json_content)
json_df.loc['limit':'total']

# default ckan API query only returns 100 rows of data
# JSON encoder returns a JSON file format/ string object 

,help,success,result
limit,https://data.sa.gov.au/data/api/3/action/help_...,True,100
records,https://data.sa.gov.au/data/api/3/action/help_...,True,"[{'_id': 1, 'ID': 1, 'School_Number': 536, 'Sc..."
records_format,https://data.sa.gov.au/data/api/3/action/help_...,True,objects
resource_id,https://data.sa.gov.au/data/api/3/action/help_...,True,86d35483-feff-42b5-ac05-ad3186ac39de
total,https://data.sa.gov.au/data/api/3/action/help_...,True,512


In [69]:
#-----identify total data records in json query-----
max_items = json_df.loc['total', 'result']  

# re-run json query with max number of data records queried

full_url = url + '&limit=' + str(max_items)
json_query = requests.get(full_url).json() 
json_content = json.dumps(json_query, indent = 1, sort_keys = False) 

df = pd.read_json(json_content)
df = pd.json_normalize(df['result']['records']) 

# normalize semi-structured JSON data into a flat table

df.head(3)

,_id,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


**Note:** An introduction to the JSON file format can be found [here](https://towardsdatascience.com/an-introduction-to-json-c9acb464f43e).  

# Examining DataFrame structure  

We can examine the properties of our DataFrame using the following functions and methods:  
+ `df.shape()` to return the DataFrame dimensions.     
+ `df.info()` to return a summary of all DataFrame columns, missing values and column object types.    
+ `df.columns` to return an index of all column names.  
+ `df.describe()` to return a simple statistical summary of each column's values.  

**Note:** You can also configure Jupyter notebook to display all columns with `pd.set_option("display.max.columns", None)`. 

In [70]:
#-----example 1.1-----  
raw_data.shape

(512, 9)

In [71]:
#-----example 1.2----- 
print("The raw data has {} rows and {} columns.".format(raw_data.shape[0], raw_data.shape[1]))

The raw data has 512 rows and 9 columns.


In [72]:
#-----example 2-----  
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 512 non-null    int64  
 1   School_Number                      512 non-null    int64  
 2   School_Name                        512 non-null    object 
 3   Suburb                             512 non-null    object 
 4   Post_Code                          512 non-null    int64  
 5   Type_of_Schooling                  512 non-null    object 
 6   Index_of_Educational_Disadvantage  512 non-null    int64  
 7   Latitude                           512 non-null    float64
 8   Longitude                          512 non-null    float64
dtypes: float64(2), int64(4), object(3)
memory usage: 36.1+ KB


In [73]:
#-----example 3-----  
raw_data.columns

Index(['ID', 'School_Number', 'School_Name', 'Suburb', 'Post_Code',
       'Type_of_Schooling', 'Index_of_Educational_Disadvantage', 'Latitude',
       'Longitude'],
      dtype='object')

In [74]:
#-----example 4-----  
raw_data.describe() # only operates on columns containing integers or floats  

,ID,School_Number,Post_Code,Index_of_Educational_Disadvantage,Latitude,Longitude
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,256.500000,771.277344,5167.619141,4.304688,-34.592120,138.502559
std,147.945936,451.007245,608.153567,1.868719,1.633955,1.529273
min,1.000000,104.000000,872.000000,1.000000,-38.002525,129.172118
25%,128.750000,388.500000,5085.750000,3.000000,-35.029207,138.505453
50%,256.500000,754.500000,5163.000000,5.000000,-34.856216,138.621977
75%,384.250000,1017.250000,5351.000000,6.000000,-34.440930,138.768058
max,512.000000,1919.000000,5734.000000,7.000000,-26.151709,140.956465


# Subsetting DataFrame rows and columns   

## Subsetting DataFrame rows  

As [previously covered](https://github.com/erikaduan/Introduction-to-Python/blob/master/03_notebooks/03_Python-data-structures.ipynb), subsetting DataFrame rows can be approached in two different ways:   

+ Using `loc` i.e. via the explicit labels-based index.  
+ Using `iloc` i.e. via the implicit 0-based index.  

In [75]:
#-----example 1----- 
raw_data.iloc[0:2+1] # subset the first 3 rows    

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [76]:
#-----example 2-----  
raw_data.iloc[0:2+1, 1] # returns the first 3 rows from the second column    

0     536
1    1673
2    1919
Name: School_Number, dtype: int64

In [77]:
#-----example 3-----  
raw_data.loc[0:2, 'School_Number'] # returns the first 3 rows from column 'School_Number'

# df.loc does not abide by the 0-index for subsetting rows  
# raw_data.loc["subject"] produces an error whereas raw_data.loc[:, "subject"] returns a subset 

0     536
1    1673
2    1919
Name: School_Number, dtype: int64

## Filtering by DataFrame rows   

We can filter by DataFrames rows in different ways:  
+ Based on variable values i.e. `df[variable_1 == 'A']`   
+ Based on a negated condition i.e. `df[variable_1 != 'A']` or `df[~ condition_A]`     
+ Using `query()` to filter by a single or multiple conditions i.e. `query('variable_1 == 'A' & year == 2002')`      
+ Filtering strings using `.str()` string methods.  

A list of Pandas DataFrame manipulation functions can be found [here](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).    

In [78]:
#-----example 1----- 
raw_data[raw_data['ID'] <= 3] # filtering by rows  

# raw_data[raw_data.ID <= 3] returns the same output as raw_data[raw_data['ID'] <= 3]  

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [79]:
#-----example 2-----
raw_data[(raw_data.ID <= 3) & (raw_data.Index_of_Educational_Disadvantage != 6)]

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [80]:
#-----example 3-----  
suburbs_list = ['Black Forest', 'Woodville']

raw_data[raw_data['Suburb'].isin(suburbs_list)]

# raw_data['Suburb'].isin(suburbs) returns a DataFrame of booleans 

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
38,39,548,Black Forest Primary School,Black Forest,5035,Primary Education,6,-34.960682,138.573308
448,449,844,The Grove Education Centre,Woodville,5011,Special Education,4,-34.873425,138.544354


**Note:** You can chain functions by enclosing code inside `()`.  

In [81]:
#-----example 4-----  
(raw_data[~ raw_data['Suburb'].isin(suburbs_list)]
.head(3)) 

# df[~ condition_A] returns the negation of condition_A

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [82]:
#-----example 5----- 
(raw_data.query('Suburb in @suburbs_list')
         .sort_values(by = ['Suburb', 'School_Name']))

# you can also use pd.query to filter conditions
# this enables us to chain Python functions to improve code readability

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
38,39,548,Black Forest Primary School,Black Forest,5035,Primary Education,6,-34.960682,138.573308
448,449,844,The Grove Education Centre,Woodville,5011,Special Education,4,-34.873425,138.544354


In [83]:
#-----example 6-----
(raw_data[raw_data['School_Name'].str.startswith('A')]
 .head(3)
)

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [84]:
#-----example 7-----  
(raw_data[raw_data['School_Name'].str.contains('High')]
 .head(3))

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
4,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678


In [85]:
#-----example 8-----  
(raw_data[raw_data['School_Name'].str.contains('.*[P|p]ark.*')]
 .head(3))

# str.contains can also evaluate regex

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
20,21,340,Ascot Park Primary School,Park Holme,5043,Primary Education,3,-34.990774,138.554376
23,24,990,Augusta Park Primary School,Port Augusta,5700,Primary Education,1,-32.495595,137.784949


## Selecting DataFrame columns   

We can select DataFrame columns in different ways:  
+ Based on column names i.e. `df[['variable_1', 'variable_2']]`  
+ By filtering column names using regular expressions i.e. `df.filter(regex = regex)`   

In [86]:
#-----example 1-----  
(raw_data[['Post_Code', 'Type_of_Schooling', 'Index_of_Educational_Disadvantage']]
.head(3))

,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage
0,5159,Primary Education,6
1,5159,Secondary Education,6
2,5000,Secondary Education,7


In [87]:
#-----example 2-----
print(type(raw_data.Suburb)) 
print(type(raw_data['Suburb']))  
print(type(raw_data[['Suburb']]))

# df[['variable_1']] returns a DataFrame but df['variable_1'] or df.variable_1 returns a Series  

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [88]:
#-----example 3-----
contains_school = df.filter(regex = '.*(S|s)chool.*')
contains_school.head(3)

# in Python, filter returns columns via regex matching

,School_Number,School_Name,Type_of_Schooling
0,536,Aberfoyle Hub R-7 School,Primary Education
1,1673,Aberfoyle Park High School,Secondary Education
2,1919,Adelaide Botanic High School,Secondary Education


To rename a column, we can use a dictionary inside the rename function i.e. `df.rename(columns = {'old_name' : 'new_name'})` or by applying string methods on `df.columns`.        

In [89]:
#-----example 1-----  
raw_data = raw_data.rename(columns = {'Index_of_Educational_Disadvantage' : 'edu_disadvantage_index',
                                      'Type_of_Schooling' : 'schooling_type'})  

raw_data.columns

Index(['ID', 'School_Number', 'School_Name', 'Suburb', 'Post_Code',
       'schooling_type', 'edu_disadvantage_index', 'Latitude', 'Longitude'],
      dtype='object')

In [90]:
#-----example 2-----
raw_data.columns = raw_data.columns.str.lower()  

raw_data.columns

Index(['id', 'school_number', 'school_name', 'suburb', 'post_code',
       'schooling_type', 'edu_disadvantage_index', 'latitude', 'longitude'],
      dtype='object')

To delete a single or multiple columns, we can use the drop function i.e. `df.drop(['variable_1', 'variable_2'])`.  

In [91]:
#-----example 3-----  
raw_data = raw_data.drop(columns = ['id'])

raw_data.columns

Index(['school_number', 'school_name', 'suburb', 'post_code', 'schooling_type',
       'edu_disadvantage_index', 'latitude', 'longitude'],
      dtype='object')

## Transforming data    

Data transformations are usually required to create new variables of interest. We can transform columns into new columns in different ways:  

+ Be applying a pre-existing function on a single or multiple columns   
+ By applying a function along an axis of the DataFrame using `apply()` i.e. `df.apply(lambda x: x.method if condition == 1 else x)`     
+ By applying a function elementwise across a DataFrame i.e. `applymap()`. 
    + Note that `applymap()` has been optimized may work much faster than `apply()` in certain cases.   
+ By substituting individual values in a Series with another value i.e. `map()`  
    + Note that `map()` accepts a dictionary, Series or function. Values that are not found in the dict are converted to NaN.  

**Note:** A [lambda function](https://realpython.com/python-lambda/) is a function that does not necessarily need to be assigned a name.  

In [92]:
#-----example 1-----  
raw_data['id'] = np.arange(1, raw_data.shape[0] + 1, 1) 
raw_data.head(3)

# create new Series using pre-defined np function

,school_number,school_name,suburb,post_code,schooling_type,edu_disadvantage_index,latitude,longitude,id
0,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142,1
1,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181,2
2,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743,3


In [93]:
#-----example 2.1-----
df_2 = pd.DataFrame(np.arange(1, 10+1).reshape(5,2),
                    columns = ['var1','var2'])
df_2 # create df_2

df_2.apply(lambda x: x ** 2)  

# by default, a lambda is applied across the whole DataFrame

,var1,var2
0,1,4
1,9,16
2,25,36
3,49,64
4,81,100


In [94]:
#-----example 2.2-----
df_2['var1_squared'] = df_2['var1'].apply(lambda x: x ** 2) 
df_2

# we can also specify individual columns to transform

,var1,var2,var1_squared
0,1,2,1
1,3,4,9
2,5,6,25
3,7,8,49
4,9,10,81


**Note:** If we want to add columns `var1` and `var2` together, we can either directly specify this transformation or define a new function and specify which axis we would like it to apply across.       

In [95]:
#-----example 2.3-----
df_2['vars_combined'] = df_2['var1'] + df_2['var2'] 
df_2

,var1,var2,var1_squared,vars_combined
0,1,2,1,3
1,3,4,9,7
2,5,6,25,11
3,7,8,49,15
4,9,10,81,19


In [96]:
#-----example 2.4-----
df_2['vars_combined'] = df_2.apply(lambda x: x['var1'] + x['var2'], axis=1) 
df_2

,var1,var2,var1_squared,vars_combined
0,1,2,1,3
1,3,4,9,7
2,5,6,25,11
3,7,8,49,15
4,9,10,81,19


**Note:** The output created by `df.apply(lambda function, axis=1, result_type='expand')` is similar to the output created using `mutate_all()` in R. 

In [97]:
#-----example 2.5-----  
def cumulative_add(df): 
    return [df['var1'] + 1, df['var2'] + 2] 

print(df_2.apply(cumulative_add, axis = 1))
print(df_2.apply(cumulative_add, axis = 1, result_type = 'expand'))

# we can assign an unnamed lambda function or named function inside apply()

0      [2, 4]
1      [4, 6]
2      [6, 8]
3     [8, 10]
4    [10, 12]
dtype: object
    0   1
0   2   4
1   4   6
2   6   8
3   8  10
4  10  12


In [98]:
#-----example 2.6-----  
df_2.applymap(lambda x: np.log2(x)) 

# applymap() does not require the axis argument and returns a scalar output for each DataFrame element 
# applying np.log2(x) across df_2 may be more computationally efficient  

,var1,var2,var1_squared,vars_combined
0,0.000000,1.000000,0.000000,1.584963
1,1.584963,2.000000,3.169925,2.807355
2,2.321928,2.584963,4.643856,3.459432
3,2.807355,3.000000,5.614710,3.906891
4,3.169925,3.321928,6.339850,4.247928


In [99]:
#-----example 2.7-----  
import timeit  

# define background setup and test code chunk

code_setup = "import numpy as np" 
  
applymap_test = ''' 
def example():  
    df_2.apply_map(lambda x: np.log2(x))  
'''

np_test = ''' 
def example():  
    df_2.np.log2()  
'''
  
# print timeit statement and compare code execution times    

print("applymap: {} seconds"
      .format(timeit.timeit(setup = code_setup, 
                            stmt = applymap_test, 
                            number = 1000000)))  

print("vectorised df: {} seconds"
      .format(timeit.timeit(setup = code_setup, 
                            stmt = np_test, 
                            number = 1000000)))  

applymap: 0.08405249999941589 seconds
vectorised df: 0.06118419999984326 seconds


**Note:** An introduction to using the `timeit` module can be found [here](https://www.geeksforgeeks.org/timeit-python-examples/). It is useful to understand when applying vectorised functions and when applying `applymap` is faster.  

In [100]:
#-----example 2.8-----
def clean_vars(var): 
    if var == 'Primary Education':   
        return 'primary'
    elif var == 'Secondary Education':
        return 'secondary'   
    elif var == 'Special Education':
        return 'special_ed'
    elif var == 'Aboriginal/Anangu Schools': 
        return 'aboriginal_anangu'
    else: 
        return np.NaN        
    
raw_data['schooling_type'].apply(clean_vars)

# clean_vars function does not work if we use var is 'Primary Education' instead of var == `Primary Education`  

0                primary
1              secondary
2              secondary
3             special_ed
4              secondary
             ...        
507              primary
508    aboriginal_anangu
509                  NaN
510                  NaN
511            secondary
Name: schooling_type, Length: 512, dtype: object

**Note:** Pandas DataFrames are built on top of Numpy arrays that use `NaN` to describe null values. To detect `NaN` values, we use `np.isnan()` when dealing with NumPy arrays and `df.isna()` or `df.isnull()` when dealing with Pandas DataFrames. This is discussed further [here](https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas).

## Aggregating data   

Aggregating data involves two main tasks:  
+ Summarising data trends  
+ Applying functions by data grouping   

### Summarising data   

To summarise data trends, we can perform different actions:  
+ By applying the method `df['var_1'].unique()` to list all unique elements in a column.  
+ By applying the method `df['var_1'].nunique()` to count the number of distinct values in a column i.e. like `n_distinct()` in R.  
+ By applying the method `df['var_1'].value_counts()` to create a count table.  
+ By applying summary methods like `mean()` or `quantile()`.  

In [101]:
#-----example 1-----
raw_data['schooling_type'].unique()

array(['Primary Education', 'Secondary Education', 'Special Education',
       'Primary/Secondary Combined', 'Aboriginal/Anangu Schools'],
      dtype=object)

In [102]:
#-----example 2-----  
raw_data['schooling_type'].nunique()

5

In [103]:
#-----example 3-----
raw_data['schooling_type'].value_counts()

Primary Education             351
Secondary Education            68
Primary/Secondary Combined     61
Aboriginal/Anangu Schools      17
Special Education              15
Name: schooling_type, dtype: int64

In [104]:
#-----example 4-----
raw_data['edu_disadvantage_index'].quantile(np.arange(0.2, 1, 0.2)) 

# applying summary methods on a column returns a Series  

0.2    2.0
0.4    4.0
0.6    5.0
0.8    6.0
Name: edu_disadvantage_index, dtype: float64

### Grouping by and aggregrating data   

To group by variables of interest:  
1. Create a GroupBy object using `df.groupby(by = 'col')`.  
2. Chain a method to the GroupBy object.  

In [148]:
#-----example 1----- 
raw_data['edu_disadvantage_index'] = raw_data['edu_disadvantage_index'].apply(lambda x: str(x))

by_schooling = (raw_data['edu_disadvantage_index']
                .groupby(raw_data['schooling_type'])
                .value_counts(normalize=True)
                .rename("proportion")
                .reset_index()) 

# .reset_index() creates a new index and converts the Series back into a DataFrame

(by_schooling
 .sort_values(['schooling_type','edu_disadvantage_index'])
 .reset_index(drop = True)
 .head(4))
 
# .reset_index(drop = True) resets the sorted DataFrame

,schooling_type,edu_disadvantage_index,proportion
0,Aboriginal/Anangu Schools,1,1.000000
1,Primary Education,1,0.051282
2,Primary Education,2,0.142450
3,Primary Education,3,0.105413


In [ ]:
#-----example 2-----  

In [149]:
#-----example 3-----  

## Handling missing data   

## Reshaping DataFrames 

## Joining datasets  